In [ ]:
!pip install -Uqq fastai

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
path = untar_data(URLs.PETS)/'images'

In [ ]:
def is_cat(filename):
    return filename[0].isupper()
catdog_path = Path('cat_or_dog')
pet_type = 'cat','dog'
for i in pet_type:
    (catdog_path / i).mkdir(parents=True, exist_ok=True)
for file in path.iterdir():
    if is_cat(file.name):
        dest = catdog_path/pet_type[0]
    else:
        dest = catdog_path/pet_type[1]
    shutil.copy(file, dest/file.name)

In [ ]:
dls = ImageDataLoaders.from_name_func(
    '.',
    get_image_files(path),
    valid_pct=0.2,
    seed=42,
    label_func=parent_label,
    item_tfms=Resize(192)
)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.load('model')

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:67: UserWarning: Saved file doesn't contain an optimizer state.
  elif with_opt: warn("Saved file doesn't contain an optimizer state.")


In [ ]:
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    label_str = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    probs_dict = {labels[i]: float(probs[i]) for i in range(len(labels))}
    return label_str, probs_dict

In [ ]:
import gradio as gr

gr.Interface(
    fn=predict,
    title = "Is it a cat?",
    description = "A cat vs dog classifier trained on the Oxford Pets dataset with fastai. Created as a demo for Gradio and HuggingFace Spaces.",
    inputs=gr.Image(height=512, width=512),
    outputs=[
        gr.Textbox(label="Prediction Summary"),
        gr.Label(num_top_classes=3)
    ],
    examples = ['siamese.webp'],
    article="<p style='text-align: center'><a href='https://tmabraham.github.io/blog/gradio_hf_spaces_tutorial' target='_blank'>Blog post</a></p>"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1a36f06e004ac2c109.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
